# OpenAPI 사양으로 함수 호출하기


인터넷의 대부분은 RESTful API로 구동됩니다. GPT에 이러한 API를 호출할 수 있는 기능을 부여하면 가능성의 세계가 열립니다. 이 노트북에서는 GPT를 사용하여 API를 지능적으로 호출하는 방법을 보여드립니다. 여기에서는 OpenAPI 사양과 연쇄 함수 호출을 활용합니다.

OpenAPI 사양(OAS)](https://swagger.io/specification/)은 기계가 읽고 해석할 수 있는 형식으로 RESTful API의 세부 사항을 설명하기 위해 보편적으로 인정되는 표준입니다. 이를 통해 인간과 컴퓨터 모두 서비스의 기능을 이해할 수 있으며, GPT에 API를 호출하는 방법을 표시하는 데 활용할 수 있습니다.

이 노트북은 두 개의 주요 섹션으로 나뉩니다:

1. 샘플 OpenAPI 사양을 채팅 완료 API의 함수 정의 목록으로 변환하는 방법.
2. 채팅 완료 API를 사용하여 사용자 지침에 따라 이러한 함수를 지능적으로 호출하는 방법.

이 절차를 진행하기 전에 [함수 호출](./How_to_call_functions_with_chat_models.ipynb)을 숙지하는 것이 좋습니다.


In [ ]:
!pip install -q jsonref # for resolving $ref's in the OpenAPI spec
!pip install -q openai

In [2]:
import os
import json
import jsonref
import openai
import requests
from pprint import pp

openai.api_key = os.environ["OPENAI_API_KEY"]

## OpenAPI 사양을 함수 정의로 변환하는 방법


여기서 사용하는 예제 OpenAPI 사양은 `gpt-4`를 사용하여 만들었습니다. 이 샘플 사양을 채팅 완료 API에 제공할 수 있는 함수 정의 집합으로 변환하겠습니다. 제공된 사용자 지침에 따라 모델은 이러한 함수를 호출하는 데 필요한 인수가 포함된 JSON 객체를 생성합니다.

계속 진행하기 전에 생성된 사양을 살펴보겠습니다. OpenAPI 사양에는 API의 엔드포인트, 지원하는 작업, 허용하는 매개변수, 처리할 수 있는 요청 및 반환하는 응답에 대한 세부 정보가 포함됩니다. 사양은 JSON 형식으로 정의됩니다.

사양의 엔드포인트에는 다음에 대한 작업이 포함됩니다:

- 모든 이벤트 나열
- 새 이벤트 생성
- ID로 이벤트 검색
- ID로 이벤트 삭제
- ID로 이벤트 이름 업데이트하기

스펙의 각 연산에는 'operationId'가 있으며, 이는 스펙을 함수 명세로 파싱할 때 함수 이름으로 사용됩니다. 사양에는 각 연산에 대한 매개변수의 데이터 유형과 구조를 정의하는 스키마도 포함되어 있습니다.

스키마는 여기에서 확인할 수 있습니다:


In [3]:
with open('./data/example_events_openapi.json', 'r') as f:
    openapi_spec = jsonref.loads(f.read()) # it's important to load with jsonref, as explained below

display(openapi_spec)

{'openapi': '3.0.0',
 'info': {'version': '1.0.0',
  'title': 'Event Management API',
  'description': 'An API for managing event data'},
 'paths': {'/events': {'get': {'summary': 'List all events',
    'operationId': 'listEvents',
    'responses': {'200': {'description': 'A list of events',
      'content': {'application/json': {'schema': {'type': 'array',
         'items': {'type': 'object',
          'properties': {'id': {'type': 'string'},
           'name': {'type': 'string'},
           'date': {'type': 'string', 'format': 'date-time'},
           'location': {'type': 'string'}},
          'required': ['name', 'date', 'location']}}}}}}},
   'post': {'summary': 'Create a new event',
    'operationId': 'createEvent',
    'requestBody': {'required': True,
     'content': {'application/json': {'schema': {'type': 'object',
        'properties': {'id': {'type': 'string'},
         'name': {'type': 'string'},
         'date': {'type': 'string', 'format': 'date-time'},
         'location

이제 OpenAPI 사양을 잘 이해했으므로 이를 함수 사양으로 파싱할 수 있습니다.

간단한 `openapi_to_functions` 함수를 작성하여 정의 목록을 생성할 수 있으며, 각 함수는 다음 키를 포함하는 사전으로 표현됩니다:

- 이름`: 이것은 OpenAPI 사양에 정의된 API 엔드포인트의 작업 식별자에 해당합니다.
- 설명`: 함수에 대한 간략한 설명 또는 요약으로, 함수가 수행하는 작업에 대한 개요를 제공합니다.
- 매개변수`: 함수에 대해 예상되는 입력 매개변수를 정의하는 스키마입니다. 각 매개변수의 유형, 필수 또는 선택 여부 및 기타 관련 세부 정보에 대한 정보를 제공합니다.

스키마에 정의된 각 엔드포인트에 대해 다음을 수행해야 합니다:

1. **JSON 참조 확인**: OpenAPI 사양에서는 중복을 피하기 위해 JSON 참조($ref라고도 함)를 사용하는 것이 일반적입니다. 이러한 참조는 여러 위치에서 사용되는 정의를 가리킵니다. 예를 들어, 여러 API 엔드포인트가 동일한 객체 구조를 반환하는 경우 해당 구조를 한 번 정의한 다음 필요한 곳에서 참조할 수 있습니다. 이러한 참조를 해결하고 참조가 가리키는 콘텐츠로 대체해야 합니다.

2. **함수 이름 추출하기:** 함수 이름으로 operationId를 간단히 사용하겠습니다. 또는 엔드포인트 경로와 작업을 함수 이름으로 사용할 수도 있습니다.

3. **설명 및 매개변수 추출하기:** `description`, `summary`, `requestBody`, `parameters` 필드를 반복하여 함수의 설명과 매개변수를 채웁니다.

구현은 다음과 같습니다:


In [4]:
def openapi_to_functions(openapi_spec):
    functions = []

    for path, methods in openapi_spec["paths"].items():
        for method, spec_with_ref in methods.items():
            # 1. Resolve JSON references.
            spec = jsonref.replace_refs(spec_with_ref)

            # 2. Extract a name for the functions.
            function_name = spec.get("operationId")

            # 3. Extract a description and parameters.
            desc = spec.get("description") or spec.get("summary", "")

            schema = {"type": "object", "properties": {}}

            req_body = (
                spec.get("requestBody", {})
                .get("content", {})
                .get("application/json", {})
                .get("schema")
            )
            if req_body:
                schema["properties"]["requestBody"] = req_body

            params = spec.get("parameters", [])
            if params:
                param_properties = {
                    param["name"]: param["schema"]
                    for param in params
                    if "schema" in param
                }
                schema["properties"]["parameters"] = {
                    "type": "object",
                    "properties": param_properties,
                }

            functions.append(
                {"name": function_name, "description": desc, "parameters": schema}
            )

    return functions


functions = openapi_to_functions(openapi_spec)

for function in functions:
    pp(function)
    print()

{'name': 'listEvents',
 'description': 'List all events',
 'parameters': {'type': 'object', 'properties': {}}}

{'name': 'createEvent',
 'description': 'Create a new event',
 'parameters': {'type': 'object',
                'properties': {'requestBody': {'type': 'object',
                                               'properties': {'id': {'type': 'string'},
                                                              'name': {'type': 'string'},
                                                              'date': {'type': 'string',
                                                                       'format': 'date-time'},
                                                              'location': {'type': 'string'}},
                                               'required': ['name',
                                                            'date',
                                                            'location']}}}}

{'name': 'getEventById',
 'description': 'Retrieve an eve

## GPT로 이러한 함수를 호출하는 방법


이제 이러한 함수 정의가 있으므로 GPT를 활용하여 사용자 입력에 따라 지능적으로 함수를 호출할 수 있습니다.

채팅 완료 API는 함수를 실행하는 것이 아니라 자체 코드에서 함수를 호출하는 데 사용할 수 있는 JSON을 생성한다는 점에 유의해야 합니다.

함수 호출에 대한 자세한 내용은 전용 [함수 호출 가이드](./How_to_call_functions_with_chat_models.ipynb)를 참조하세요.


In [14]:
SYSTEM_MESSAGE = """
You are a helpful assistant. 
Respond to the following prompt by using function_call and then summarize actions. 
Ask for clarification if a user request is ambiguous.
"""

# Maximum number of function calls allowed to prevent infinite or lengthy loops
MAX_CALLS = 5


def get_openai_response(functions, messages):
    return openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k-0613",
        functions=functions,
        function_call="auto",  # "auto" means the model can pick between generating a message or calling a function.
        temperature=0,
        messages=messages,
    )


def process_user_instruction(functions, instruction):
    num_calls = 0
    messages = [
        {"content": SYSTEM_MESSAGE, "role": "system"},
        {"content": instruction, "role": "user"},
    ]

    while num_calls < MAX_CALLS:
        response = get_openai_response(functions, messages)
        message = response["choices"][0]["message"]

        if message.get("function_call"):
            print(f"\n>> Function call #: {num_calls + 1}\n")
            pp(message["function_call"])
            messages.append(message)

            # For the sake of this example, we'll simply add a message to simulate success.
            # Normally, you'd want to call the function here, and append the results to messages.
            messages.append(
                {
                    "role": "function",
                    "content": "success",
                    "name": message["function_call"]["name"],
                }
            )

            num_calls += 1
        else:
            print("\n>> Message:\n")
            print(message["content"])
            break

    if num_calls >= MAX_CALLS:
        print(f"Reached max chained function calls: {MAX_CALLS}")


USER_INSTRUCTION = """
Instruction: Get all the events. 
Then create a new event named AGI Party.
Then delete event with id 2456.
"""

process_user_instruction(functions, USER_INSTRUCTION)


 >> Function call #: 1

{'name': 'listEvents',
 'arguments': '{}'}

 >> Function call #: 2

{'name': 'createEvent',
 'arguments': '{\n'
              '  "requestBody": {\n'
              '    "id": "1234",\n'
              '    "name": "AGI Party",\n'
              '    "date": "2022-12-31",\n'
              '    "location": "New York"\n'
              '  }\n'
              '}'}

 >> Function call #: 3

{'name': 'deleteEvent',
 'arguments': '{\n  "parameters": {\n    "id": "2456"\n  }\n}'}

>> Message:

Actions summary:
1. Retrieved all the events successfully.
2. Created a new event named "AGI Party" with ID 1234, scheduled for December 31, 2022, in New York.
3. Deleted the event with ID 2456.


결론 ### 결론

지금까지 OpenAPI 사양을 함수 사양으로 변환하여 GPT가 이를 지능적으로 호출할 수 있도록 하는 방법과 이를 서로 연결하여 복잡한 연산을 수행하는 방법을 보여드렸습니다.

이 시스템의 가능한 확장에는 조건부 로직이나 루핑이 필요한 더 복잡한 사용자 지시를 처리하고, 실제 API와 통합하여 실제 연산을 수행하고, 오류 처리 및 유효성 검사를 개선하여 지시가 실행 가능하고 함수 호출이 성공하도록 하는 것이 포함될 수 있습니다.
